# Comparision between # complaints and # incidents in Miami, FL carrier companies

In [1]:
pip install pandas psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import psycopg2 as pg

### Create engine and dataframes from uploaded tables¶
- fmcsa_companies
- fmcsa_safer_data

In [3]:
engine = pg.connect("dbname='clever' user='clever' host='postgres_clever' port='5432' password='clever'")

companies_df = pd.read_sql('select * from public.fmcsa_companies', con=engine)
safer_data_df = pd.read_sql('select * from public.fmcsa_safer_data', con=engine)

/tmp/ipykernel_7374/2199064119.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  companies_df = pd.read_sql('select * from public.fmcsa_companies', con=engine)
/tmp/ipykernel_7374/2199064119.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  safer_data_df = pd.read_sql('select * from public.fmcsa_safer_data', con=engine)


### Check on columns on both dataframes and select only desired ones

In [4]:
companies_df.columns

Index(['usdot_num', 'company_name', 'company_url', 'city', 'state',
       'total_complaints_2021', 'total_complaints_2022',
       'total_complaints_2023', 'location', 'company_type'],
      dtype='object')

In [5]:
safer_data_df.columns

Index(['usdot_num', 'entity_type', 'operating_status', 'legal_name',
       'physical_address', 'phone', 'drivers', 'operation_classification',
       'carrier_type', 'cargo_types', 'us_vehicle_inspections',
       'us_driver_inspections', 'us_vehicle_out_of_service',
       'us_driver_out_of_service', 'us_crashes_fatal', 'us_crashes_injury',
       'carrier_safety_rating_rating_date',
       'carrier_safety_rating_review_date', 'carrier_safety_rating_rating',
       'carrier_safety_rating_type', 'mileage', 'mileage_year'],
      dtype='object')

In [6]:
companies_df = companies_df[['usdot_num', 'company_name', 'city', 'state', 'total_complaints_2021', 'total_complaints_2022', 'total_complaints_2023']]
safer_data_df = safer_data_df[['usdot_num', 'operating_status', 'us_crashes_injury','us_crashes_fatal']]

### Analyse crashes per company

- `us_crashes_injury`
- `us_crashes_fatal`

In [7]:
crashes_df = companies_df.merge(safer_data_df, on='usdot_num', how='inner', sort=True)
crashes_df.head()

,usdot_num,company_name,city,state,total_complaints_2021,total_complaints_2022,total_complaints_2023,operating_status,us_crashes_injury,us_crashes_fatal
0,8257,"HANSEN BROS TRANSFER & STORAGE CO., INC.",SEATTLE,WA,0,1,1,AUTHORIZED FOR Property,0,0
1,12103,MITCHELL MOVING & STORAGE COMPANY,SEATTLE,WA,0,0,0,AUTHORIZED,0,0
2,290691,DELIVERY LIMITED,DALLAS,TX,0,0,0,AUTHORIZED FOR Property,0,0
3,318974,"GRAHAM TRUCKING, INC.",SEATTLE,WA,0,0,0,AUTHORIZED FOR Property,2,0
4,430728,LLBJ TRUCKING LLC,MIAMI,FL,0,0,0,"AUTHORIZED FOR Property, HHG",0,0


### Get # of total complaints in the last 3 years

In [8]:
crashes_df['total_complaints'] = crashes_df['total_complaints_2021'] + crashes_df['total_complaints_2022'] + crashes_df['total_complaints_2023']
crashes_df = crashes_df[['usdot_num', 'company_name', 'city', 'state', 'total_complaints', 'us_crashes_injury','us_crashes_fatal', 'operating_status']]
crashes_df.head()

,usdot_num,company_name,city,state,total_complaints,us_crashes_injury,us_crashes_fatal,operating_status
0,8257,"HANSEN BROS TRANSFER & STORAGE CO., INC.",SEATTLE,WA,2,0,0,AUTHORIZED FOR Property
1,12103,MITCHELL MOVING & STORAGE COMPANY,SEATTLE,WA,0,0,0,AUTHORIZED
2,290691,DELIVERY LIMITED,DALLAS,TX,0,0,0,AUTHORIZED FOR Property
3,318974,"GRAHAM TRUCKING, INC.",SEATTLE,WA,0,2,0,AUTHORIZED FOR Property
4,430728,LLBJ TRUCKING LLC,MIAMI,FL,0,0,0,"AUTHORIZED FOR Property, HHG"


### The 10 carrier companies with most crashes and # of complaints

In [9]:
miami_crashes_df = crashes_df[(crashes_df['city'] == 'MIAMI') & (crashes_df['state'] == 'FL')]
miami_crashes_df.sort_values(['us_crashes_injury','us_crashes_fatal', 'total_complaints'], ascending=False).head(10)

,usdot_num,company_name,city,state,total_complaints,us_crashes_injury,us_crashes_fatal,operating_status
33,1452661,D & M TRUCKING SERVICES CORP,MIAMI,FL,0,2,0,"AUTHORIZED FOR Property, HHG"
37,1475638,FLAT RATE MOVING AND STORAGE LLC,MIAMI,FL,1,1,0,AUTHORIZED FOR HHG
136,2805350,ETG MOVING & DELIVERY LLC,MIAMI,FL,0,1,0,AUTHORIZED FOR Property
13,1151019,TOP 10 VAN LINES INC,MIAMI,FL,16,0,1,AUTHORIZED FOR HHG
189,3307068,US STANDARD MOVING & STORAGE CORP,MIAMI,FL,72,0,0,NOT AUTHORIZED
95,2247863,MOVING APT INC,MIAMI,FL,46,0,0,"AUTHORIZED FOR BROKER Property, HHG"
227,3491781,EASY ROAD MOVING & STORAGE INC,MIAMI,FL,30,0,0,AUTHORIZED FOR HHG
138,2821065,A1A MOVERS LLC,MIAMI,FL,24,0,0,AUTHORIZED FOR HHG
142,2886867,MOVING SERVICES,MIAMI,FL,21,0,0,AUTHORIZED FOR BROKER HHG
120,2543958,JACOB AND SONS MOVING & STORAGE,MIAMI,FL,19,0,0,AUTHORIZED FOR HHG
